In [72]:
import nbimporter

### Simulator Backend

In [73]:
from qiskit import Aer
from qiskit.utils import QuantumInstance, algorithm_globals

seed = 50
algorithm_globals.random_seed = seed

backend = Aer.get_backend('qasm_simulator')

### BFGS Optimizer

In [74]:
from qiskit.algorithms.optimizers import L_BFGS_B

bfgs_optimizer = L_BFGS_B(maxiter=60)

### Variational Quantum Circuit

In [92]:
from ansatz_circuit_item2 import *

thetas = np.random.uniform(low=0, high=2*np.pi, size=8)
phis = np.random.uniform(low=0, high=2*np.pi, size=4)

get_full_variational_quantum_circuit(thetas=thetas,phis=phis,D1=2,D2=6)

AttributeError: 'QuantumCircuit' object has no attribute 'composed'

In [59]:
import numpy as np

import nbimporter
import anzats_SSVQE_article as anzats

class ExpvalCost:
    """
    Create a cost function that gives the expectation value of an input Hamiltonian.

    """

    def __init__(
        self,
        hamiltonian,
        backend,
    ):
        self.hamiltonian = hamiltonian
        self.backend = backend
        
        def cost_fn(theta_params, phi_params, basis, D1, D2, weighted=False):
            cost = 0
            for state_vector in basis:
                qc = anzats.anzats_circ(theta_params, phi_params, D1, D2)
                qc.initialize(state_vector)

                qobj = assemble(qc)
                state = sim.run(qobj).result().get_statevector()
                
                dagger_state = np.matrix(state)
                state = dagger_state.getH()
                
                state = np.array(state)
                dagger_state = np.array(dagger_state)
                
                exp_val = np.matmul(dagger_state, np.matmul(self.hamiltonian, state))

                if weighted and basis[state_vector]==(len(basis)-1):
                    cost += 0.5*exp_val
                else:
                    cost += exp_val
                
                return cost
            
        self.cost_function = cost_fn

    def __call__(self, params, state_idx):
        return self.cost_function(params, state_idx)


In [60]:
ham = np.identity(16)
single_cost = ExpvalCost(ham, backend)

In [63]:
def get_k_basis(k, n):
    full_basis = np.identity(n)
    return full_basis[:k]

In [62]:
num_qubits = 2
w = np.arange(num_qubits, 0, -1)

In [64]:
max_iterations = 200
costs = []

# Initial parameters for the ansatz
num_layers = 8
params = np.random.uniform(low=0, high=2*np.pi, size=(num_layers, num_qubits, 3))

In [71]:
Ising_callback_energies = []
Ising_callback_energies_single_0 = []
Ising_callback_energies_single_1 = []

for _ in range(max_iterations):
    params, prev_energy = bfgs_optimizer.optimize(12,single_cost)
    energy = total_cost(params)
    energy_ground_state = single_cost(params, state_idx=0)
    energy_first_excited_state = single_cost(params, state_idx=1)
    Ising_callback_energies_single_0.append(energy_ground_state)
    Ising_callback_energies_single_1.append(energy_first_excited_state)
    Ising_callback_energies.append(energy)

ValueError: Initial point is required but None given